In [1]:
import pandas as pd
from datasets import create_datalists, create_datasets
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
from tpot import TPOTRegressor

In [2]:
configs = {
    'pretrain': 'None',
    'img_size': 512,
    'model': 'Resnet18',
    'epochs': 100,
    'batch_size': 32,
    'weight_decay': 1e-3,
    'lr': 1e-4,
    'min_lr': 0.000006463,
    'RandomHorizontalFlip': True,
    'RandomVerticalFlip': True,
    'RandomRotation': True,
    'ZoomIn': False,
    'ZoomOut': False,
    'use_mix': False,
    'use_avg': False,
    'XShift': False,
    'YShift': False,
    'RandomShear': False,
    'max_shear': 30,  # value in degrees
    'max_shift': 0.5,
    'rotation_angle': 3,
    'zoomin_factor': 0.95,
    'zoomout_factor': 0.05,
}

imgs_list, label_list = create_datalists()

final_train = True

train_dataset, test_dataset = create_datasets(imgs_list, label_list, configs, final_train=final_train, patients_out=True, patient_ids_out=[1, 2, 3])

In [3]:
X_train_convert = np.array([x.numpy() for x, l in train_dataset])
y_train_convert = np.array([l.numpy() for x, l in train_dataset])

In [4]:
y_train_convert.shape

(1000,)

In [5]:
X_train_convert = np.reshape(X_train_convert, (1000, 262144))

In [6]:
X_train_convert.shape

(1000, 262144)

In [7]:
df = pd.DataFrame(X_train_convert)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,262134,262135,262136,262137,262138,262139,262140,262141,262142,262143
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024249,0.068283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

X_train, X_test, y_train, y_test = train_test_split(df, y_train_convert, test_size=0.2, random_state=42)


knn_regressor = KNeighborsRegressor(n_neighbors=30)
knn_regressor.fit(X_train, y_train)

y_pred = knn_regressor.predict(X_test)

def validate(y_pred, y_test):
    aggregate_results = dict()
    # aggregate_results["plcc"] = abs(pearsonr(y_pred, y_test)[0])
    # aggregate_results["srocc"] = abs(spearmanr(y_pred, y_test)[0])
    # aggregate_results["krocc"] = abs(kendalltau(y_pred, y_test)[0])
    aggregate_results["overall"] = abs(pearsonr(y_pred, y_test)[0]) + abs(
        spearmanr(y_pred, y_test)[0]) + abs(kendalltau(y_pred, y_test)[0])
    print(aggregate_results)
    mse = np.mean((y_pred - y_test) ** 2)
    print(f"Mean Squared Error: {mse}")

In [ ]:
# Create and fit TPOTRegressor with standard MSE scoring
tpot = TPOTRegressor(scoring='neg_mean_squared_error', generations=10, population_size=50, random_state=42, verbosity=3)
tpot.fit(X_train, y_train)

30 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/550 [00:00<?, ?pipeline/s]

Skipped pipeline #5 due to time out. Continuing to the next pipeline.
Skipped pipeline #7 due to time out. Continuing to the next pipeline.
Skipped pipeline #12 due to time out. Continuing to the next pipeline.
Skipped pipeline #14 due to time out. Continuing to the next pipeline.
Skipped pipeline #16 due to time out. Continuing to the next pipeline.
